In [1]:
import spacy
import pandas as pd
import en_core_web_md
import ast

In [2]:
# load medium English model in case we need to work with vectors
nlp = en_core_web_md.load()

In [3]:
df = pd.read_csv('../Word2Vec/subneighborhood_separated_articles/2018.csv')

In [4]:
#black_neighborhoods = ['dorchester', 'roxbury', 'mattapan', 'hyde_park']
#white_neighborhoods = ['fenway', 'beacon_hill', 'downtown', 'south_boston', 'east_boston', 'back_bay', 'jamaica_plain',
#                      'south_end', 'charlestown', 'brighton', 'allston', 'west_end', 'roslindale', 'north_end',
#                      'mission_hill', 'harbor_islands', 'longwood_medical_area', 'west_roxbury']
#df = df.fillna("('no article', 'no_id')")
#df['dorchester'] = df['dorchester'].apply(ast.literal_eval)
#df['roxbury'] = df['roxbury'].apply(ast.literal_eval)
#df['mattapan'] = df['mattapan'].apply(ast.literal_eval)
#df['hyde_park'] = df['hyde_park'].apply(ast.literal_eval)
#df['fenway'] = df['fenway'].apply(ast.literal_eval)
#df['beacon_hill'] = df['beacon_hill'].apply(ast.literal_eval)
#df['downtown'] = df['downtown'].apply(ast.literal_eval)
#df['south_boston'] = df['south_boston'].apply(ast.literal_eval)
#df['east_boston'] = df['east_boston'].apply(ast.literal_eval)
#df['back_bay'] = df['back_bay'].apply(ast.literal_eval)
#df['jamaica_plain'] = df['jamaica_plain'].apply(ast.literal_eval)
#df['south_end'] = df['south_end'].apply(ast.literal_eval)
#df['charlestown'] = df['charlestown'].apply(ast.literal_eval)
#df['brighton'] = df['brighton'].apply(ast.literal_eval)
#df['allston'] = df['allston'].apply(ast.literal_eval)
#df['west_end'] = df['west_end'].apply(ast.literal_eval)
#df['roslindale'] = df['roslindale'].apply(ast.literal_eval)
#df['north_end'] = df['north_end'].apply(ast.literal_eval)
#df['mission_hill'] = df['mission_hill'].apply(ast.literal_eval)
#df['harbor_islands'] = df['harbor_islands'].apply(ast.literal_eval)
#df['longwood_medical_area'] = df['longwood_medical_area'].apply(ast.literal_eval)
#df['west_roxbury'] = df['west_roxbury'].apply(ast.literal_eval)

In [5]:
black_sub_neighborhoods = ['wellington_hill', 'southern_mattapan', 'west_codman_hill', 'west_lower_mills', 'washington_hill',
                          'mattapan', 'franklin_field_south', 'dorchester_center', 'dorchester', 'bowdoin_north', 'mount_bowdoin',
                          'west_street', 'river_street', 'dudley', 'brunswick_king', 'nubian_square', 'roxbury', 'grove_hall',
                          'codman_square', 'upham\'s_corner', 'meeting_house_hill', 'washington_park', 'hyde_park', 'jones_hill',
                          'milton_hill', 'forest_hills', 'woodbourne', 'lower_roxbury', 'fields_corner_wester',
                          'frederick_douglas_square_historic_district', 'georgetown', 'st._marks', 'fort_hill', 'ashcroft',
                          'ashmount', 'clam_point', 'neponset', 'port_norfolk', 'columbus_park', 
                          'andrew_square', 'adams_village']

white_sub_neighborhoods = ['metropolitan_hill', 'beech_street', 'washington', 'mount_hope', 'd_street', 'west_broadway', 
                          'central_maverick_square', 'paris_square', 'bunker_hill', 'high_street_hill', 'roslindale',
                          'eagle_hill', 'columbus', 'central_village', 'east_boston',
                          'shawmut', 'harbor_view', 'orient_heights', 'mission_hill', 'south_end', 'jeffries_point', 
                          'upper_washington', 'spring_street', 'charlestown', 'stonybrook', 'bay_village', 'city_point',
                          'jamaica_plain', 'back_bay', 'brighton', 'egleston_square', 'west_fens', 'allston', 'lower_allston',
                          'hunnwell_hill', 'salisbury_road-corey_farm', 'boston', 'fenway-kenmore', 'oak_square', 
                          'medford_street', 'the_neck', 'brook_farm', 'seaport_district', 'centre_south',
                          'st._elizabeth\'s', 'china_town', 'aberdeen', 'bellevue_hill', 'downtown_crossing', 
                          'chestnut_hill_neighborhood_association', 'back_bay_west', 'west_end', 'telegraph_hill', 
                          'beachmont']

df = df.fillna("('no article', 'no_id')")
df = df.drop(['Unnamed: 0'], axis=1)
for col in df.columns:
    df[col] = df[col].apply(ast.literal_eval)

In [6]:
col

'beachmont'

In [7]:
cols = df.columns
for col in df.columns:
    if df.iloc[0][col] == ('no article', 'no_id'):
        df = df.drop([col], axis=1)

rem_cols = df.columns

In [8]:
articles = {}
for col in rem_cols:
    articles[col] = []

for sub in rem_cols:
    for i in range(df.shape[0]):
        if df.loc[i][sub] != ('no article', 'no_id'):
            articles[sub].append((nlp(df.loc[i, sub][0]), df.loc[i, sub][1]))
    print(sub + ' DONE')

mattapan DONE
dorchester DONE
west_street DONE
dudley DONE
roxbury DONE
grove_hall DONE
codman_square DONE
hyde_park DONE
neponset DONE
washington DONE
bunker_hill DONE
columbus DONE
boston DONE
the_neck DONE


In [9]:
df.shape

(736, 14)

In [10]:
#articles = {'hyde_park': [], 'beacon_hill': [], 'south_boston': [], 'jamaica_plain': [], 'east_boston': [],
#                'south_end': [], 'back_bay': [], 'north_end': [], 'west_roxbury': [], 'mission_hill': [],
#                'harbor_islands': [], 'west_end': [], 'longwood_medical_area': [],
#                'dorchester': [], 'roxbury': [], 'downtown': [], 'fenway': [], 'mattapan': [], 'brighton': [],
#                'charlestown': [], 'roslindale': [], 'allston': []}
#for sub_neighborhood in articles.keys():
#    for i in range(df.shape[0]):
#        if type(df.loc[i, sub_neighborhood]) == tuple:
#            articles[sub_neighborhood].append((nlp(df.loc[i, sub_neighborhood][0]), df.loc[i, sub_neighborhood][1]))
#    print(sub_neighborhood + ' DONE')

In [11]:
articles['dorchester'][0][1]

'2018_1904'

In [12]:
people = {}
for col in rem_cols:
    people[col] = []
    
for sub_neighborhood in articles.keys():
    for (doc, article_id) in articles[sub_neighborhood]:
        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                name = ent[0:2]
                # sentence = ent.sent
                people[sub_neighborhood].append((name, article_id))

In [13]:
#people = {'hyde_park': [], 'beacon_hill': [], 'south_boston': [], 'jamaica_plain': [], 'east_boston': [],
#                'south_end': [], 'back_bay': [], 'north_end': [], 'west_roxbury': [], 'mission_hill': [],
#                'harbor_islands': [], 'west_end': [], 'longwood_medical_area': [],
#                'dorchester': [], 'roxbury': [], 'downtown': [], 'fenway': [], 'mattapan': [], 'brighton': [],
#                'charlestown': [], 'roslindale': [], 'allston': []}
#
#for sub_neighborhood in articles.keys():
#    for (doc, article_id) in articles[sub_neighborhood]:
#        for ent in doc.ents:
#            if ent.label_ == 'PERSON':
#                name = ent[0:2]
#                # sentence = ent.sent
#                people[sub_neighborhood].append((name, article_id))

In [14]:
for sub_neighborhood in people.keys():
    list1 = people[sub_neighborhood]
    # convert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    people[sub_neighborhood] = unique_list

In [15]:
for sub_neighborhood in people.keys():
    for i in range(len(people[sub_neighborhood])):
        if people[sub_neighborhood][i][0].text.strip() != '':
            temp = people[sub_neighborhood][i][0].text.split()
            if len(temp) > 1:
                people[sub_neighborhood][i] = (temp[-1], people[sub_neighborhood][i][1])
            else:
                people[sub_neighborhood][i] = (temp[0], people[sub_neighborhood][i][1])

In [16]:
people['dorchester'][0]

('Mary', '2018_2151')

In [28]:
from ethnicolr import pred_wiki_ln

In [29]:
# temp = pd.DataFrame(people['fenway'], columns=['last_name', 'article_id'])
# temp = temp.drop(['article_id'], axis=1)
# temp['subneighborhood'] = 'fenway'
# temp.head()
# people['dorchester']

In [31]:
final_df = pd.DataFrame(columns=['last_name', 'subneighborhood'])
subs = list(rem_cols)
#subs.remove('longwood_medical_area')
for col in subs:
    temp = pd.DataFrame(people[col], columns=['last_name', 'article_id'])
    temp = temp.drop(['article_id'], axis=1)
    temp['subneighborhood'] = col
    preds = pred_wiki_ln(temp, 'last_name')
    final_df = pd.concat([final_df, preds], axis=0)
    print(col + ' DONE')

mattapan DONE
dorchester DONE
west_street DONE
dudley DONE
roxbury DONE
grove_hall DONE
codman_square DONE
hyde_park DONE
neponset DONE
washington DONE
bunker_hill DONE
columbus DONE
boston DONE
the_neck DONE


In [32]:
hen = final_df['race'].value_counts()
pd.DataFrame(hen)

,race
"GreaterEuropean,British",16460
"Asian,GreaterEastAsian,EastAsian",2094
"GreaterEuropean,WestEuropean,French",1762
"GreaterEuropean,WestEuropean,Hispanic",1622
"GreaterEuropean,WestEuropean,Italian",1494
"GreaterEuropean,Jewish",1332
"Asian,IndianSubContinent",564
"GreaterAfrican,Muslim",505
"GreaterEuropean,EastEuropean",361
"GreaterEuropean,WestEuropean,Nordic",263


In [33]:
final_df = final_df.drop(['Asian,GreaterEastAsian,EastAsian', 'Asian,GreaterEastAsian,Japanese', 'Asian,IndianSubContinent', 'GreaterAfrican,Africans', 'GreaterAfrican,Muslim', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean', 'GreaterEuropean,Jewish', 'GreaterEuropean,WestEuropean,French','GreaterEuropean,WestEuropean,Germanic','GreaterEuropean,WestEuropean,Hispanic','GreaterEuropean,WestEuropean,Italian','GreaterEuropean,WestEuropean,Nordic'], axis=1)

for i in range(len(final_df.index)):
    if final_df.iloc[i]['race'] == 'Asian,GreaterEastAsian,EastAsian' or final_df.iloc[i]['race'] == 'Asian,GreaterEastAsian,Japanese' or final_df.iloc[i]['race'] == 'Asian,IndianSubContinent':
        final_df.iloc[i]['race'] = 'api'
    elif final_df.iloc[i]['race'] == 'GreaterEuropean,WestEuropean,Hispanic':
        final_df.iloc[i]['race'] = 'hispanic'
    elif final_df.iloc[i]['race'] == 'GreaterAfrican,Muslim' or final_df.iloc[i]['race'] == 'GreaterAfrican,Africans':
        final_df.iloc[i]['race'] = 'black'
    else:
        final_df.iloc[i]['race'] = 'white'

final_df

,last_name,subneighborhood,race
0,Yancey,mattapan,white
1,Labossiere,mattapan,white
2,Davis,mattapan,white
3,Eddins,mattapan,white
4,Hilaire,mattapan,white
...,...,...,...
38,Flanagan,the_neck,white
39,Reynolds,the_neck,white
40,Flanagan,the_neck,white
41,Briggs,the_neck,white


In [34]:
groups = final_df.groupby(final_df.subneighborhood)
race_df = pd.DataFrame(columns=subs)
races = final_df.race.unique()
for col in subs:
    group = groups.get_group(col)
    race_df[col] = group.race.value_counts()

In [35]:
race_df = race_df.fillna(0.0)
race_df

,mattapan,dorchester,west_street,dudley,roxbury,grove_hall,codman_square,hyde_park,neponset,washington,bunker_hill,columbus,boston,the_neck
white,832,6186,6.0,316,1958,12.0,2.0,520,3.0,394,35.0,62.0,11508,40.0
api,112,623,0.0,40,219,4.0,0.0,94,0.0,47,2.0,0.0,1701,3.0
hispanic,31,596,0.0,10,224,0.0,0.0,4,0.0,5,1.0,2.0,749,0.0
black,24,149,0.0,3,68,2.0,0.0,27,0.0,6,0.0,0.0,367,0.0


In [36]:
for i in range(len(subs)):
    tot = race_df[subs[i]].sum()
    for j in race_df.index:
        race_df.loc[j, [subs[i]]] = (race_df.loc[j, [subs[i]]] / tot) * 100

In [37]:
race_df

,mattapan,dorchester,west_street,dudley,roxbury,grove_hall,codman_square,hyde_park,neponset,washington,bunker_hill,columbus,boston,the_neck
white,83.283283,81.890389,100.0,85.636856,79.303362,66.666667,100.0,80.620155,100.0,87.168142,92.105263,96.875,80.335079,93.023256
api,11.211211,8.247286,0.0,10.840108,8.869988,22.222222,0.0,14.573643,0.0,10.398230,5.263158,0.000,11.874346,6.976744
hispanic,3.103103,7.889860,0.0,2.710027,9.072499,0.000000,0.0,0.620155,0.0,1.106195,2.631579,3.125,5.228621,0.000000
black,2.402402,1.972465,0.0,0.813008,2.754151,11.111111,0.0,4.186047,0.0,1.327434,0.000000,0.000,2.561955,0.000000


In [38]:
#race_df.index = ['white', 'hispanic', 'black', 'asian']
race_df

,mattapan,dorchester,west_street,dudley,roxbury,grove_hall,codman_square,hyde_park,neponset,washington,bunker_hill,columbus,boston,the_neck
white,83.283283,81.890389,100.0,85.636856,79.303362,66.666667,100.0,80.620155,100.0,87.168142,92.105263,96.875,80.335079,93.023256
api,11.211211,8.247286,0.0,10.840108,8.869988,22.222222,0.0,14.573643,0.0,10.398230,5.263158,0.000,11.874346,6.976744
hispanic,3.103103,7.889860,0.0,2.710027,9.072499,0.000000,0.0,0.620155,0.0,1.106195,2.631579,3.125,5.228621,0.000000
black,2.402402,1.972465,0.0,0.813008,2.754151,11.111111,0.0,4.186047,0.0,1.327434,0.000000,0.000,2.561955,0.000000


In [39]:
race_df.to_csv('Wiki_Last_Name_Race_Representation_by_Sub_Neighborhood/2018_representation.csv')

In [23]:
#agg_df = pd.DataFrame(columns=subs, index=['white', 'black', 'api', 'hispanic'])

In [24]:
#agg_df = agg_df.fillna(0.0)

In [25]:
#pred_races = race_df.index
#for sub in subs:
#    for each in pred_races:
#        temp = race_df.loc[each][sub]
#        if each == 'GreaterEuropean,WestEuropean,Hispanic':
#            agg_df.loc['hispanic'][sub] += temp
#        elif each == 'Asian,GreaterEastAsian,EastAsian' or each == 'Asian,IndianSubContinent' or each == 'Asian,GreaterEastAsian,Japanese':
#            agg_df.loc['api'][sub] += temp
#        elif each == 'GreaterAfrican,Muslim' or each == 'GreaterAfrican,Africans':
#            agg_df.loc['black'][sub] += temp
#        else:
#            agg_df.loc['white'][sub] += temp

In [26]:
#agg_df.to_csv('Wiki_Full_Name_Race_Representation_by_Neighborhood/2018_representation.csv')

In [27]:
#agg_df